# Usefull

In [ ]:
#all_df_sample = all_df.sample(frac=0.0003, replace=True)
#all_df.dtypes
#df.shape

### Фильтрация уникальных записей в датасете по значениям в столбце

In [305]:
df = pd.DataFrame([['aaa', 123], ['aaa', 111], ['bbb', 456], ['aaa', 123], ['ddd', 456]], columns=['f', 's'])
# col_names = ['f', 's']
# df.columns = col_names
df

,f,s
0,aaa,123
1,aaa,111
2,bbb,456
3,aaa,123
4,ddd,456


Ищет целиком повторяющиеся строки и удаляет их - не то что нужно

In [306]:
# grouped = dfа.groupby(col_names)
# index = [gp_keys[0] for gp_keys in grouped.groups.values()]
# unique_df = df.reindex(index)

In [307]:
def rm_users_with_same_bid(df):
    s_uniq = df['s'].unique()
    
    for i in s_uniq:
        row_idx_tb_rm = []
        flag = True
        for index, row in df.iterrows():
            if (row['s'] == i) & flag:
                cur_bid = row['s']
                cur_uid = row['f']
                flag = False            
            if (row['s'] == cur_bid) & (row['f'] != cur_uid):
                row_idx_tb_rm.append(index)
                
        # print('iteration#{}: {}'.format(i, row_idx_tb_rm))

        df.drop(row_idx_tb_rm, inplace = True)
        
    df.reset_index(inplace=True, drop=True)
    
    return df

In [308]:
rm_users_with_same_bid(df)

,f,s
0,aaa,123
1,aaa,111
2,bbb,456
3,aaa,123


In [309]:
def rm_users_with_multiple_bid(df):
    f_uniq = df['f'].unique()
    
    for i in f_uniq:        
        row_idx_tb_rm = []
        flag = True
        for idx, row in df.iterrows():    
            if (row['f'] == i) & flag:
                cur_bid = row['s']
                cur_uid = row['f']
                flag = False
            if (row['f'] == cur_uid) & (row['s'] != cur_bid):
                row_idx_tb_rm.append(idx)
                
        df.drop(row_idx_tb_rm, inplace = True)
    
    # обновим индексы
    df.reset_index(inplace=True, drop=True)
    
    return df        

In [310]:
rm_users_with_multiple_bid(df)

,f,s
0,aaa,123
1,bbb,456
2,aaa,123


In [185]:
for idx, row in df.iterrows():
    print(row)
    

f    aaa
s    123
Name: 0, dtype: object
f    bbb
s    456
Name: 1, dtype: object
f    aaa
s    123
Name: 2, dtype: object
f    ddd
s    456
Name: 3, dtype: object


#### Потестим

In [188]:
#!pip install pytest
# import pytest

In [316]:
def test_no_diff_users_with_same_bid():
    col_names = ['f', 's']
    # тест на сохранность
    df = pd.DataFrame([['a', 1], 
                       ['b', 2], 
                       ['c', 3]], columns=col_names)
    
    # тест на удаление и на индексы
    df2 = pd.DataFrame([['a', 1], 
                       ['c', 1],
                       ['b', 2], 
                       ['d', 2]], columns=col_names)
    
    df2_after = pd.DataFrame([['a', 1], ['b', 2]], columns=col_names)
    
    assert rm_users_with_same_bid(df).equals(df)
    assert rm_users_with_same_bid(df2).equals(df2_after)
        
    
test_no_diff_users_with_same_bid()    

In [317]:
def test_no_user_with_multiple_bid():
    col_names = ['f', 's']
    # тест на сохранность
    df = pd.DataFrame([['a', 1], ['b', 2], ['c', 3]], columns=col_names)
    
    df2 = pd.DataFrame([['a', 1],
                       ['a', 2],
                       ['a', 1],
                       ['a', 1]], columns=col_names)
    
    # тест на удаление и на индексы
    df2_after = pd.DataFrame([['a',1], ['a',1], ['a',1]], columns=col_names)
    
    assert rm_users_with_multiple_bid(df).equals(df)
    assert rm_users_with_multiple_bid(df2).equals(df2_after)
    
    
test_no_user_with_multiple_bid()    

### Uniq по первому uniq по второму

In [4]:
import pandas as pd

In [14]:
df = pd.DataFrame([['a', 1], ['a', 2], ['a', 3]], columns=['f', 's'])

In [15]:
df

,f,s
0,a,1
1,a,2
2,a,3


In [ ]:
df['s'].distinct()

In [3]:
s = pd.Series([1,2,3])

In [4]:
s

0    1
1    2
2    3
dtype: int64

In [7]:
s[s == 2]

1    2
dtype: int64

### Filter example

In [2]:
import pandas as pd

In [3]:
s = pd.Series([1,2,3,4,5])
s2 = pd.Series([2,4])

In [6]:
s[~s.isin(s2)]

0    1
2    3
4    5
dtype: int64

#### Проверим что отрезки ts двух df пересекаются

In [35]:
bots = pd.Series([1,2,3,4])
http = pd.DataFrame([[1, 'a'],[2, 'b'],[3, 'c'],[4, 'd'],[5, 'e'], [6, 'f']], columns=['uid', 'bid'])

In [36]:
http

,uid,bid
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e
5,6,f


In [37]:
bots

0    1
1    2
2    3
3    4
dtype: int64

In [46]:
http['bot_or_human'] = http['uid'].apply(lambda x: 1 if x in bots.values else 0)

In [47]:
http

,uid,bid,bot_or_human
0,1,a,1
1,2,b,1
2,3,c,1
3,4,d,1
4,5,e,0
5,6,f,0


In [45]:
4 in bots.values

True